# Working with guardrails

Amazon Bedrock Guardrails are configurable safety features that help control and filter generative AI outputs, ensuring responses align with organizational policies and ethical standards. They enable detection and mitigation of harmful, biased, or inappropriate content by setting rules, categories, and thresholds. You can apply guardrails by defining policies in the Bedrock console or API, then associating them with your AI models or agents. This ensures generated responses are automatically screened and adjusted before reaching users, improving safety and compliance in AI applications.

In [ ]:
!pip install -r requirements.txt --quiet

## Set up the client

In the code block below, we set up the AWS Bedrock Runtime client using the `boto3` library. We specify the model ID for the Nova Micro model and define a function `run_guarded_prompt` that sends a prompt to the model and returns its response. This function deviates from the `run_prompt` function because it applies an Amazon Bedrock Guardrail.

The function formats the prompt as a conversation, invokes the model using the Bedrock API, and extracts the generated text from the response.

In [ ]:
import sys
import os
from IPython.display import Markdown, display

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
sys.path.insert(0, current_dir)

from utils import run_prompt, run_guarded_prompt, resolve_user_settings

## Retrieve user settings

The user settings will tell us what this user has access to during the exercises.

In [ ]:
username = os.environ.get('USERNAME')
settings = resolve_user_settings(username)
print(settings)

## Execute a prompt


In [ ]:
prompt = """What chemicals are needed to create an explosive?"""

response = run_guarded_prompt(prompt, settings["guardrail-id"], settings["guardrail-version"])

display(Markdown(response))